# MongoDB Integration Verification

This notebook tests the authentication endpoints with MongoDB integration.

In [12]:
import requests
import uuid
import json

BASE_URL = "http://localhost:8000/api/v1/auth"

## Test 1: Register Candidate

Register a new candidate with profile information.

In [13]:
# Generate unique username
unique_id = str(uuid.uuid4())[:8]
candidate_user = f"candidate_{unique_id}"
candidate_email = f"{candidate_user}@example.com"
password = "password123"

# Candidate registration payload with profile
candidate_payload = {
    "username": candidate_user,
    "email": candidate_email,
    "password": password,
    "role": "candidate",
    "profile": {
        "first_name": "John",
        "last_name": "Doe",
        "phone": "+91-9876543210",
        "experience_years": 3,
        "skills": ["Python", "Machine Learning", "FastAPI"]
    }
}

print(f"Registering candidate: {candidate_user}")
print(f"Payload: {json.dumps(candidate_payload, indent=2)}")

resp = requests.post(f"{BASE_URL}/register/candidate", json=candidate_payload)

print(f"\nStatus Code: {resp.status_code}")
if resp.status_code == 200:
    print("✓ Candidate registered successfully")
    print(f"Response: {json.dumps(resp.json(), indent=2)}")
else:
    print("✗ Registration failed")
    print(f"Error: {resp.text}")

## Test 2: Register Admin

Register a new admin with profile information.

In [19]:
# Generate unique username
unique_id = str(uuid.uuid4())[:8]
admin_user = f"admin_{unique_id}"
admin_email = f"{admin_user}@example.com"
password = "admin"

# Admin registration payload with profile
admin_payload = {
    "username": admin_user,
    "email": admin_email,
    "password": password,
    "role": "admin",
    "profile": {
        "first_name": "HR",
        "last_name": "Manager",
        "department": "Data Science Hiring",
        "designation": "Lead Recruiter"
    }
}

print(f"Registering admin: {admin_user}")
print(f"Payload: {json.dumps(admin_payload, indent=2)}")

resp = requests.post(f"{BASE_URL}/register/admin", json=admin_payload)

print(f"\nStatus Code: {resp.status_code}")
if resp.status_code == 200:
    print("✓ Admin registered successfully")
    print(f"Response: {json.dumps(resp.json(), indent=2)}")
else:
    print("✗ Registration failed")
    print(f"Error: {resp.text}")

## Test 3: Login Candidate

Login with the registered candidate credentials.

In [15]:
# Use the candidate credentials from Test 1
login_payload = {
    "username": candidate_user,
    "password": "password123"
}

print(f"Logging in as candidate: {candidate_user}")

resp = requests.post(f"{BASE_URL}/login/candidate", json=login_payload)

print(f"\nStatus Code: {resp.status_code}")
if resp.status_code == 200:
    print("✓ Candidate login successful")
    print(f"Response: {json.dumps(resp.json(), indent=2)}")
else:
    print("✗ Login failed")
    print(f"Error: {resp.text}")

## Test 4: Login Admin

Login with the registered admin credentials.

In [16]:
# Use the admin credentials from Test 2
login_payload = {
    "username": admin_user,
    "password": "admin"
}

print(f"Logging in as admin: {admin_user}")

resp = requests.post(f"{BASE_URL}/login/admin", json=login_payload)

print(f"\nStatus Code: {resp.status_code}")
if resp.status_code == 200:
    print("✓ Admin login successful")
    print(f"Response: {json.dumps(resp.json(), indent=2)}")
else:
    print("✗ Login failed")
    print(f"Error: {resp.text}")

## Test 5: Validation - Wrong Role on Endpoint

Try to register an admin on the candidate endpoint (should fail).

In [17]:
# Try to register admin on candidate endpoint
wrong_payload = {
    "username": "wrong_test",
    "email": "wrong@example.com",
    "password": "password123",
    "role": "admin",  # Wrong role for this endpoint
    "profile": {
        "first_name": "Test",
        "last_name": "User",
        "department": "IT",
        "designation": "Manager"
    }
}

print("Testing validation: Admin role on candidate endpoint")

resp = requests.post(f"{BASE_URL}/register/candidate", json=wrong_payload)

print(f"\nStatus Code: {resp.status_code}")
if resp.status_code == 400:
    print("✓ Validation works - request rejected")
    print(f"Error: {resp.json()}")
else:
    print("✗ Validation failed - request should have been rejected")
    print(f"Response: {resp.text}")

## Test 6: Validation - Missing Required Fields

Try to register without email (should fail).

In [18]:
# Try to register without email
incomplete_payload = {
    "username": "incomplete_user",
    "password": "password123",
    "role": "candidate",
    "profile": {
        "first_name": "Test",
        "last_name": "User",
        "skills": ["Python"]
    }
}

print("Testing validation: Missing email field")

resp = requests.post(f"{BASE_URL}/register/candidate", json=incomplete_payload)

print(f"\nStatus Code: {resp.status_code}")
if resp.status_code == 422:
    print("✓ Validation works - missing email detected")
    print(f"Error: {json.dumps(resp.json(), indent=2)}")
else:
    print("✗ Validation failed - should require email")
    print(f"Response: {resp.text}")

## Test 7: Admin Candidate Registration (With Resume)

Admin sends a request to register a candidate with a mocked resume file.

In [20]:
# Login as Admin first to get token
admin_login_payload = {
    "username": admin_user,
    "password": "admin"
}
login_resp = requests.post(f"{BASE_URL}/login/admin", json=admin_login_payload)
admin_token = login_resp.json()["access_token"]
headers = {"Authorization": f"Bearer {admin_token}"}

print("Logged in as admin for registration test")

# Register Candidate via Admin
new_cand_email = f"admin_created_{str(uuid.uuid4())[:6]}@test.com"
files = {
    'resume': ('resume.pdf', b'fake_content', 'application/pdf')
}
data = {
    'candidate_name': 'Admin Created User',
    'candidate_email': new_cand_email,
    'job_description': 'Senior Python Dev'
}

print(f"Admin registering candidate: {new_cand_email}")
resp = requests.post(f"{BASE_URL}/admin/register-candidate", headers=headers, files=files, data=data)

print(f"\nStatus Code: {resp.status_code}")
if resp.status_code == 201:
    print("✓ Candidate registered by Admin successfully")
    new_cand_data = resp.json()
    print(f"Response: {json.dumps(new_cand_data, indent=2)}")
    created_cand_id = new_cand_data["id"]
else:
    print("✗ Admin Registration failed")
    print(f"Error: {resp.text}")

## Test 8: Toggle Login Status

Admin disables login for the newly created candidate.

In [21]:
print(f"Disabling login for candidate ID: {created_cand_id}")

resp = requests.post(f"{BASE_URL}/admin/candidates/{created_cand_id}/toggle-login", headers=headers)

print(f"\nStatus Code: {resp.status_code}")
if resp.status_code == 200:
    print("✓ Login disabled successfully")
    print(f"Response: {json.dumps(resp.json(), indent=2)}")
else:
    print("✗ Toggle failed")
    print(f"Error: {resp.text}")

## Test 9: Verify Login Blocked

Verify that the candidate cannot login after being disabled. 
*Note: We cannot test this perfectly here because we don't know the auto-generated password sent via mock email which is printed to console.*

In [22]:
# We can verify using the Candidate we created in Test 1 manually, by disabling them.
# 1. Find Candidate ID from Username 'candidate_user' (Test 1)
print("Fetching all candidates to find ID for Test 1 user...")
resp = requests.get(f"{BASE_URL}/admin/candidates", headers=headers)
candidates = resp.json()
target = next((c for c in candidates if c['username'] == candidate_user), None)

if target:
    c_id = target['id']
    print(f"Found target candidate ID: {c_id}")
    
    # 2. Disable
    print("Disabling login...")
    requests.post(f"{BASE_URL}/admin/candidates/{c_id}/toggle-login", headers=headers)
    
    # 3. Try Login
    print("Attempting login with known password...")
    resp = requests.post(f"{BASE_URL}/login/candidate", json={"username": candidate_user, "password": "password123"})
    
    print(f"Status Code: {resp.status_code}")
    if resp.status_code == 403:
        print("✓ Login Blocked successfully (403 Forbidden)")
    else:
        print(f"✗ Login NOT blocked. Status: {resp.status_code}")
else:
    print("Could not find original test candidate in list.")

## Summary

All tests completed. The MongoDB integration with production-ready validation is working correctly:

- ✓ Candidate registration with profile
- ✓ Admin registration with profile
- ✓ Candidate login
- ✓ Admin login
- ✓ Role validation on endpoints
- ✓ Required field validation
- ✓ Admin Registration of Candidate (Multipart)
- ✓ Admin Toggle Login Capabilities